## Read News Data (US removed)

In [36]:
import pandas as pd

sentiments = pd.read_parquet('https://github.com/DaiZack/CAhousing/blob/master/housing_result_ca.parquet?raw=true')
sentiments['quarter'] = sentiments['datetime'].dt.to_period('Q')
sentiments = sentiments.groupby(['city','quarter']).aggregate({'title':'count', 'wordscount':'sum', 'sentiment':'sum'}).reset_index()
sentiments['score'] = sentiments.sentiment/sentiments.wordscount
sentiments.tail()

,city,quarter,title,wordscount,sentiment,score
586,Winnipeg,2018Q4,8,4046,85,0.021008
587,Winnipeg,2019Q1,4,3054,35,0.011460
588,Winnipeg,2019Q2,1,1206,9,0.007463
589,Winnipeg,2019Q3,3,1466,28,0.019100
590,Winnipeg,2019Q4,1,777,10,0.012870


## House Index

In [37]:
houseIndex = pd.read_csv('https://github.com/DaiZack/CAhousing/raw/master/House_Price_Index.csv')
houseIndex = houseIndex.drop('c11', axis=1)
houseIndex['quarter'] = pd.to_datetime(houseIndex['Transaction Date'],format='%b-%y').dt.to_period('Q')
houseIndex = houseIndex.groupby(['quarter']).mean().reset_index()
houseIndex = houseIndex.melt(id_vars=['quarter'],var_name='city', value_name='houseIndex')
# houseIndex['deltaP'] = houseIndex.houseIndex.rolling(2).apply(lambda x:np.log(x[1]+1)-np.log(x[0]+1))
houseIndex.head()

,quarter,city,houseIndex
0,2000Q1,Vancouver,67.733333
1,2000Q2,Vancouver,68.343333
2,2000Q3,Vancouver,68.623333
3,2000Q4,Vancouver,68.050000
4,2001Q1,Vancouver,68.220000


## Fundamental



1.   Monthly Indexes CPI, Rent, Employment, indexes Index



In [38]:
import numpy as np
indexes = pd.read_csv('https://github.com/DaiZack/CAhousing/raw/master/indexesAll.csv')
indexes.drop('Employment rate', axis=1, inplace=True)
indexes.rename(columns={'Unemployment rate':'Unemployment'}, inplace=True)
indexes['quarter'] = pd.to_datetime(indexes.date, format='%Y-%m-%d').dt.to_period('Q')
indexes = indexes.groupby(['quarter']).mean().reset_index()
deltaX = indexes.rolling(2).apply(lambda x:np.log(x[1]+1)-np.log(x[0]+1))
deltaX['quarter'] = indexes['quarter']
deltaX.tail()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  import sys


,CPI,Unemployment,Mortage,Rent,GDP,quarter
75,0.002716,-0.115999,0.010388,0.003530,0.001045,2018Q4
76,0.003938,0.139529,0.006061,0.013730,0.001362,2019Q1
77,0.008317,-0.092523,-0.042586,0.001069,0.008352,2019Q2
78,0.004133,0.030153,-0.036601,0.009569,0.004166,2019Q3
79,0.003874,-0.082521,-0.001964,0.008954,-0.000407,2019Q4


In [39]:
for i in range(2,3):
  for col in ['CPI',	'Unemployment',	'Mortage',	'Rent',	'GDP']:
    deltaX[col+'_'+str(i-1)] = fData[col].rolling(i).apply(lambda x: x[0])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  This is separate from the ipykernel package so we can avoid doing imports until


## Fundamental Model

In [40]:
fData = houseIndex.merge(deltaX, on='quarter', how='left')


fData.head(7)

,quarter,city,houseIndex,CPI,Unemployment,Mortage,Rent,GDP,CPI_1,Unemployment_1,Mortage_1,Rent_1,GDP_1
0,2000Q1,Vancouver,67.733333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000Q2,Vancouver,68.343333,0.004534,-0.075035,0.002140,0.003085,0.009521,NaN,NaN,NaN,NaN,NaN
2,2000Q3,Vancouver,68.623333,0.009006,0.017168,-0.026351,0.003075,0.009637,0.004534,-0.075035,0.002140,0.003085,0.009521
3,2000Q4,Vancouver,68.050000,0.010633,-0.052414,-0.015110,0.003746,0.005616,0.009006,0.017168,-0.026351,0.003075,0.009637
4,2001Q1,Vancouver,68.220000,0.002385,0.138006,-0.062832,0.003732,0.001935,0.010633,-0.052414,-0.015110,0.003746,0.005616
5,2001Q2,Vancouver,68.753333,0.012851,-0.056240,-0.008737,0.004055,0.001908,0.002385,0.138006,-0.062832,0.003732,0.001935
6,2001Q3,Vancouver,69.676667,0.000336,0.008230,-0.007608,0.005046,-0.000231,0.012851,-0.056240,-0.008737,0.004055,0.001908


In [41]:
import statsmodels.formula.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices

R2s = {}

for city in list(set(fData.city)):
  cityFdata = fData.loc[fData.city==city,:]
  cityFdata['deltaP'] = cityFdata.houseIndex.rolling(2).apply(lambda x: np.log(x[1]+1)-np.log(x[0]+1))
  for i in range(2,6):
    cityFdata['deltaP'+'_'+str(i-1)] = cityFdata['deltaP'].rolling(i).apply(lambda x: x[0])

  cityFdata.dropna(inplace=True)
  formula = 'deltaP~'+ '+'.join([c for c in cityFdata.columns if '_' in c])
  model = sm.ols(formula = formula, data=cityFdata).fit()
  y, X = dmatrices(formula, data = cityFdata, return_type='dataframe')
  vif=pd.DataFrame()
  vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
  vif["features"] = X.columns
  print('\n--------------------------',city,'-------------',len(cityFdata),'-------------\n')
  print(model.summary())
  print(vif)
  R2s[city]= model.rsquared_adj

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep 


-------------------------- Calgary ------------- 75 -------------

                            OLS Regression Results                            
Dep. Variable:                 deltaP   R-squared:                       0.546
Model:                            OLS   Adj. R-squared:                  0.483
Method:                 Least Squares   F-statistic:                     8.684
Date:                Fri, 31 Jan 2020   Prob (F-statistic):           2.00e-08
Time:                        04:50:10   Log-Likelihood:                 190.01
No. Observations:                  75   AIC:                            -360.0
Df Residuals:                      65   BIC:                            -336.8
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------

In [42]:
print(R2s)
print('average Adjust-R-Squre:',np.mean(list(R2s.values())))

{'Calgary': 0.48309178429619304, 'Toronto': 0.2882268539950903, 'Edmonton': 0.6384907834670438, 'Vancouver': 0.42736712823021317, 'Winnipeg': 0.49948397606672623, 'Hamilton': 0.4180223138145822, 'Montreal': 0.5243512373766381, 'Ottawa': 0.636443090773028}
average Adjust-R-Squre: 0.48943464600243936


## Sentment Model

In [43]:
sData = houseIndex.merge(deltaX, on='quarter', how='left')
sData = sData.merge(sentiments, on=['quarter','city'], how='left')
# for i in range(2,6):
#   for col in ['CPI',	'Unemployment',	'Mortage',	'Rent',	'GDP']:
#     sData[col+'_'+str(i-1)] = sData[col].rolling(i).apply(lambda x: x[0])

sData.head(7)

,quarter,city,houseIndex,CPI,Unemployment,Mortage,Rent,GDP,CPI_1,Unemployment_1,Mortage_1,Rent_1,GDP_1,title,wordscount,sentiment,score
0,2000Q1,Vancouver,67.733333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000Q2,Vancouver,68.343333,0.004534,-0.075035,0.002140,0.003085,0.009521,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000Q3,Vancouver,68.623333,0.009006,0.017168,-0.026351,0.003075,0.009637,0.004534,-0.075035,0.002140,0.003085,0.009521,NaN,NaN,NaN,NaN
3,2000Q4,Vancouver,68.050000,0.010633,-0.052414,-0.015110,0.003746,0.005616,0.009006,0.017168,-0.026351,0.003075,0.009637,NaN,NaN,NaN,NaN
4,2001Q1,Vancouver,68.220000,0.002385,0.138006,-0.062832,0.003732,0.001935,0.010633,-0.052414,-0.015110,0.003746,0.005616,NaN,NaN,NaN,NaN
5,2001Q2,Vancouver,68.753333,0.012851,-0.056240,-0.008737,0.004055,0.001908,0.002385,0.138006,-0.062832,0.003732,0.001935,NaN,NaN,NaN,NaN
6,2001Q3,Vancouver,69.676667,0.000336,0.008230,-0.007608,0.005046,-0.000231,0.012851,-0.056240,-0.008737,0.004055,0.001908,1.0,963.0,13.0,0.013499


In [44]:
nR2s = {}

for city in list(set(sData.city)):
  citySdata = sData.loc[sData.city==city,:]
  citySdata['deltaP'] = citySdata.houseIndex.rolling(2).apply(lambda x: np.log(x[1]+1)-np.log(x[0]+1))
  for col in ['deltaP', 'score']:
    for i in range(2,6):
      citySdata[col+'_'+str(i-1)] = citySdata[col].rolling(i).apply(lambda x: x[0])
  cityFdata.dropna(inplace=True)
  formula = 'deltaP~'+ '+'.join([c for c in citySdata.columns if '_' in c and 'Unemployment' not in c])
  model = sm.ols(formula = formula, data=citySdata).fit()
  y, X = dmatrices(formula, data = citySdata, return_type='dataframe')
  vif=pd.DataFrame()
  vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
  vif["features"] = X.columns
  print('\n--------------------------',city,'-------------',len(cityFdata),'-------------\n')
  print(model.summary())
  print(vif)
  nR2s[city] = model.rsquared_adj

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=Fa


-------------------------- Calgary ------------- 75 -------------

                            OLS Regression Results                            
Dep. Variable:                 deltaP   R-squared:                       0.567
Model:                            OLS   Adj. R-squared:                  0.476
Method:                 Least Squares   F-statistic:                     6.224
Date:                Fri, 31 Jan 2020   Prob (F-statistic):           7.46e-07
Time:                        04:50:11   Log-Likelihood:                 176.77
No. Observations:                  70   AIC:                            -327.5
Df Residuals:                      57   BIC:                            -298.3
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [45]:
print(nR2s)
print(np.mean(list(nR2s.values())))

{'Calgary': 0.47601504558115504, 'Toronto': 0.34666429977848134, 'Edmonton': 0.5651902974894969, 'Vancouver': 0.5590573108733172, 'Winnipeg': 0.4655131833804709, 'Hamilton': 0.43032639167753384, 'Montreal': 0.47107571434390105, 'Ottawa': 0.7149287416446888}
0.5035963730961306
